In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('TwitterSentimentAnalysisDataset.csv',encoding='windows-1252',names=['Target','ID','Date','Flag','User','Text'])

Exploratory Data Analysis

In [3]:
df.shape

(1600000, 6)

In [4]:
df.head()

,Target,ID,Date,Flag,User,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [5]:
df.sample(n=10,random_state=15)

,Target,ID,Date,Flag,User,Text
561460,0,2205426244,Wed Jun 17 04:42:48 PDT 2009,NO_QUERY,BurntCaramel,@mjsluijs Nah.. Most of the antivirus program...
1570185,4,2188543728,Mon Jun 15 22:05:33 PDT 2009,NO_QUERY,gab_iii,@zacseif fly to arizona and we'll have video g...
58096,0,1685820960,Sun May 03 02:15:21 PDT 2009,NO_QUERY,NoSheds,@stephenfry I think you killed mobypic - Site ...
283874,0,1993027493,Mon Jun 01 09:42:50 PDT 2009,NO_QUERY,carebearrr,@mileycyrus and we're gonna miss you guys so ...
34009,0,1564669484,Mon Apr 20 04:29:22 PDT 2009,NO_QUERY,timaree,so tired this morning
801000,4,1468005634,Mon Apr 06 23:13:16 PDT 2009,NO_QUERY,MerCleve,@matt_the_great forget the rest...You need to ...
1066153,4,1965220208,Fri May 29 15:32:16 PDT 2009,NO_QUERY,jemcsh,"@tommcfly ur prince harry came to the usa, so ..."
796932,0,2327967353,Thu Jun 25 09:00:32 PDT 2009,NO_QUERY,evestirling,"@DrJoolz thanks, I went there before...it does..."
1345228,4,2043955672,Fri Jun 05 08:39:37 PDT 2009,NO_QUERY,jclanger,change of plans sims party tonight and then t...
415064,0,2060975737,Sat Jun 06 19:56:30 PDT 2009,NO_QUERY,Valv30,@Twisted4Jordan I am sad bout that. But this ...


In [6]:
df.columns

Index(['Target', 'ID', 'Date', 'Flag', 'User', 'Text'], dtype='object')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   Target  1600000 non-null  int64 
 1   ID      1600000 non-null  int64 
 2   Date    1600000 non-null  object
 3   Flag    1600000 non-null  object
 4   User    1600000 non-null  object
 5   Text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [8]:
df['Flag'].nunique()

1

In [9]:
df = df.drop('Flag',axis=1)

In [10]:
df

,Target,ID,Date,User,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [11]:
df.dtypes

Target     int64
ID         int64
Date      object
User      object
Text      object
dtype: object

In [12]:
df['Date'].head()

0    Mon Apr 06 22:19:45 PDT 2009
1    Mon Apr 06 22:19:49 PDT 2009
2    Mon Apr 06 22:19:53 PDT 2009
3    Mon Apr 06 22:19:57 PDT 2009
4    Mon Apr 06 22:19:57 PDT 2009
Name: Date, dtype: object

In [13]:
df['Date']

0          Mon Apr 06 22:19:45 PDT 2009
1          Mon Apr 06 22:19:49 PDT 2009
2          Mon Apr 06 22:19:53 PDT 2009
3          Mon Apr 06 22:19:57 PDT 2009
4          Mon Apr 06 22:19:57 PDT 2009
                       ...             
1599995    Tue Jun 16 08:40:49 PDT 2009
1599996    Tue Jun 16 08:40:49 PDT 2009
1599997    Tue Jun 16 08:40:49 PDT 2009
1599998    Tue Jun 16 08:40:49 PDT 2009
1599999    Tue Jun 16 08:40:50 PDT 2009
Name: Date, Length: 1600000, dtype: object

In [14]:
df['Date'] = df['Date'].str.replace(' PDT 2009','')

In [15]:
df.head()

,Target,ID,Date,User,Text
0,0,1467810369,Mon Apr 06 22:19:45,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
for ind, row in df.iterrows():
    d = row['Date']
    df.loc[ind,['Day']] = d[:3]

In [ ]:
df